In [2]:
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()
#hide
from fastai.vision.all import *
from fastbook import *

matplotlib.rc('image', cmap='Greys')

(#2) [Path('training'),Path('testing')]

(#10) [Path('training/8'),Path('training/3'),Path('training/9'),Path('training/2'),Path('training/4'),Path('training/1'),Path('training/0'),Path('training/5'),Path('training/7'),Path('training/6')]

In [7]:
modell = nn.Sequential(
    nn.Linear(28*28,30),
    nn.ReLU(),
    nn.Linear(30,10),
    nn.CrossEntropyLoss
    


torch.nn.modules.container.Sequential

In [12]:
from torchvision import datasets

In [13]:
datasets.MNIST

torchvision.datasets.mnist.MNIST

In [14]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")


Using cuda device


In [152]:
class NossonNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.layers = nn.Sequential(
            nn.Linear(28*28, 1024), nn.ReLU(),
            nn.Linear(1024, 512), nn.ReLU(),
            nn.Linear(512, 10)
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.layers(x)
        return logits


In [153]:
model = NossonNet().to(device)

In [154]:
print(model)

NossonNet(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (layers): Sequential(
    (0): Linear(in_features=784, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [155]:
loss_fn = nn.CrossEntropyLoss()

In [156]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [157]:
train

<function __main__.train(dataloader, model, loss_fn, optimizer)>

In [158]:
from torch.utils.data import DataLoader, Subset

In [159]:
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

In [160]:
from torchvision.transforms import ToTensor

In [161]:
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [162]:
bs = 64
train_dl = DataLoader(training_data, batch_size=(bs))
test_dl = DataLoader(test_data, batch_size=(bs))

In [163]:
for X, y in test_dl:
    print(f"x shape: {X.shape}")
    print(f"y shape: {y.shape} {y.dtype}")
    break

x shape: torch.Size([64, 1, 28, 28])
y shape: torch.Size([64]) torch.int64


In [164]:
training_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [165]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        pred = model.forward(X)
        loss = loss_fn(pred, y)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [166]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y)
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

            

In [167]:
epochs = 5


In [169]:
for t in range(epochs):
    print(f"Epoch: {t+1} \n-------------")
    train(train_dl, model, loss_fn, optimizer)
    test(test_dl, model, loss_fn)

Epoch: 1 
-------------
loss: 0.009298  [   64/60000]
loss: 0.030736  [ 6464/60000]
loss: 0.091963  [12864/60000]
loss: 0.012878  [19264/60000]
loss: 0.007566  [25664/60000]
loss: 0.011539  [32064/60000]
loss: 0.032311  [38464/60000]
loss: 0.010416  [44864/60000]
loss: 0.124058  [51264/60000]
loss: 0.041994  [57664/60000]
Test Error: 
 Accuracy: 96.5%, Avg loss: 0.153824 

Epoch: 2 
-------------
loss: 0.063908  [   64/60000]
loss: 0.000682  [ 6464/60000]
loss: 0.002297  [12864/60000]
loss: 0.001522  [19264/60000]
loss: 0.062076  [25664/60000]
loss: 0.003467  [32064/60000]
loss: 0.016062  [38464/60000]
loss: 0.018538  [44864/60000]
loss: 0.135696  [51264/60000]
loss: 0.011569  [57664/60000]
Test Error: 
 Accuracy: 97.8%, Avg loss: 0.103558 

Epoch: 3 
-------------
loss: 0.000543  [   64/60000]
loss: 0.003368  [ 6464/60000]
loss: 0.007537  [12864/60000]
loss: 0.090774  [19264/60000]
loss: 0.002959  [25664/60000]
loss: 0.028009  [32064/60000]
loss: 0.002309  [38464/60000]
loss: 0.002656

In [178]:
training_data[0][1]

5

In [181]:
nn.Flatten()(training_data[0][0]).shape

torch.Size([1, 784])

In [220]:
train_subset = Subset(training_data, list(range(50)))
test_subset = Subset(test_data, list(range(50)))
subset_train_loader = DataLoader(train_subset, batch_size=(10))
subset_test_loader = DataLoader(test_subset, batch_size=(10))

In [221]:
coopppy = NossonNet().to(device)

In [222]:
opt = torch.optim.Adam(coopppy.parameters(), lr=1e-3)

In [223]:
for t in range(epochs*10):
    print(f"Epoch: {t+1} \n-------------")
    train(subset_train_loader, coopppy, loss_fn, opt)
    test(subset_test_loader, coopppy, loss_fn)

Epoch: 1 
-------------
loss: 2.308925  [   10/   50]
Test Error: 
 Accuracy: 38.0%, Avg loss: 2.069172 

Epoch: 2 
-------------
loss: 1.733873  [   10/   50]
Test Error: 
 Accuracy: 54.0%, Avg loss: 1.652073 

Epoch: 3 
-------------
loss: 0.994937  [   10/   50]
Test Error: 
 Accuracy: 68.0%, Avg loss: 1.230214 

Epoch: 4 
-------------
loss: 0.417016  [   10/   50]
Test Error: 
 Accuracy: 68.0%, Avg loss: 0.979632 

Epoch: 5 
-------------
loss: 0.149245  [   10/   50]
Test Error: 
 Accuracy: 68.0%, Avg loss: 0.887896 

Epoch: 6 
-------------
loss: 0.030401  [   10/   50]
Test Error: 
 Accuracy: 62.0%, Avg loss: 0.946310 

Epoch: 7 
-------------
loss: 0.008176  [   10/   50]
Test Error: 
 Accuracy: 64.0%, Avg loss: 1.041824 

Epoch: 8 
-------------
loss: 0.003167  [   10/   50]
Test Error: 
 Accuracy: 64.0%, Avg loss: 1.081735 

Epoch: 9 
-------------
loss: 0.001262  [   10/   50]
Test Error: 
 Accuracy: 62.0%, Avg loss: 1.093410 

Epoch: 10 
-------------
loss: 0.000637  [   1